In [32]:
import numpy as np
import pandas as pd

#from sklearn import cross_validation as CV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, PolynomialFeatures
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, RobustScaler, StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF

from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import accuracy_score as accuracy

train = pd.read_csv('./data/original_train.csv')
#testa = pd.read_csv('./data/original_test_A.csv')
testb = pd.read_csv('./data/original_test_B.csv')
# valid = pd.read_csv('./adversarial_set_data.csv')
# valid = valid.drop( [ 'TARGET', 'predictions' ], axis = 1 )

remain_col = ['AGN_CNT_RCT_12_MON',
 'NB_RCT_3_MON_LGN_TMS_AGV',
 'AGN_AGR_LATEST_AGN_AMT',
 'MON_12_EXT_SAM_TRSF_IN_AMT',
 'MON_12_EXT_SAM_TRSF_OUT_AMT',
 'MON_12_EXT_SAM_NM_TRSF_OUT_CNT',
 'MON_12_EXT_SAM_AMT',
 'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT',
 'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT',
 'MON_12_TRX_AMT_MAX_AMT_PCTT',
 'MON_12_AGV_TRX_CNT',
 'MON_12_ACM_ENTR_ACT_CNT',
 'MON_12_AGV_ENTR_ACT_CNT',
 'MON_12_ACM_LVE_ACT_CNT',
 'MON_12_AGV_LVE_ACT_CNT',
 'MON_6_50_UP_ENTR_ACT_CNT',
 'MON_6_50_UP_LVE_ACT_CNT',
 'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY',
 'MON_12_ACT_IN_50_UP_CNT_PTY_QTY',
 'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
 'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV',
 'LAST_12_MON_MON_AVG_TRX_AMT_NAV',
 'REG_CPT',
 'HLD_DMS_CCY_ACT_NBR',
 'REG_DT',
 'HLD_FGN_CCY_ACT_NBR']
# 73['AGN_CNT_RCT_12_MON', 'COR_KEY_PROD_HLD_NBR', 'LAST_12_MON_COR_DPS_DAY_AVG_BAL', 'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL', 'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV', 'LAST_12_MON_MON_AVG_TRX_AMT_NAV', 'MON_12_ACM_ENTR_ACT_CNT', 'MON_12_ACT_IN_50_UP_CNT_PTY_QTY', 'MON_12_AGV_ENTR_ACT_CNT', 'MON_12_EXT_SAM_AMT', 'MON_12_EXT_SAM_TRSF_OUT_AMT', 'MON_6_50_UP_ENTR_ACT_CNT', 'NB_RCT_3_MON_LGN_TMS_AGV', 'OPN_TM', 'REG_DT']
# 76 ['COR_KEY_PROD_HLD_NBR', 'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT', 'CUR_YEAR_COUNTER_ENCASH_CNT', 'EMP_NBR', 'HLD_DMS_CCY_ACT_NBR', 'HLD_FGN_CCY_ACT_NBR', 'LAST_12_MON_COR_DPS_DAY_AVG_BAL', 'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL', 'LAST_12_MON_MON_AVG_TRX_AMT_NAV', 'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY', 'MON_12_AGV_ENTR_ACT_CNT', 'MON_12_EXT_SAM_TRSF_IN_AMT', 'MON_6_50_UP_ENTR_ACT_CNT', 'NB_RCT_3_MON_LGN_TMS_AGV', 'REG_DT']
# train = train.drop(need_drop_col + my_guess, axis=1)
# test = test.drop(need_drop_col + my_guess, axis=1)
train = train[['CUST_UID','LABEL'] + remain_col]
test = testb[['CUST_UID'] + remain_col]
#test = testb
print(train.shape,test.shape)

# import copy
# train = copy.deepcopy(train)
# test = copy.deepcopy(test)
train['TARGET'] = 0
test['TARGET'] = 1


data = pd.concat(( train, test ))

# shuffle
data = data.iloc[ np.random.permutation(len( data )) ]
data.reset_index( drop = True, inplace = True )
cat_cols = ['MON_12_CUST_CNT_PTY_ID','AI_STAR_SCO','WTHR_OPN_ONL_ICO',
'SHH_BCK','LGP_HLD_CARD_LVL','NB_CTC_HLD_IDV_AIO_CARD_SITU'] 
question_cols = ['REG_CPT','CUR_YEAR_MID_BUS_INC']
large_cale_cols = ['ICO_CUR_MON_ACM_TRX_AMT',
'COUNTER_CUR_YEAR_CNT_AMT',
'PUB_TO_PRV_TRX_AMT_CUR_YEAR',
'MON_12_EXT_SAM_TRSF_IN_AMT',
'MON_12_EXT_SAM_TRSF_OUT_AMT',
'MON_12_EXT_SAM_AMT',
'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT',
'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT',
'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL',
'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV',
'LAST_12_MON_MON_AVG_TRX_AMT_NAV'
]

large_offset  = ['AGN_CUR_YEAR_AMT',
'AGN_CUR_YEAR_WAG_AMT',
'PUB_TO_PRV_TRX_AMT_CUR_YEAR',
'CUR_YEAR_MID_BUS_INC'
]

for col in cat_cols:
    if col in data.columns:
        data = data.drop( [ col] , axis = 1 )
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
data = clean_inf_nan(data)
data = data.fillna(-1)
x = data.drop( [ 'TARGET', 'CUST_UID', 'LABEL' ] , axis = 1 )
y = data.TARGET

train_examples = len(data)-12000
x_train = x[:train_examples]
x_test = x[train_examples:]
y_train = y[:train_examples]
y_test = y[train_examples:]

#

print ("LR...")

clf = LR()
clf.fit( x_train, y_train )

# predict

p = clf.predict_proba( x_test )[:,1]
auc = AUC( y_test, p )
print ("AUC: {:.2%}".format( auc ))

# AUC: 50.06%

print ("RF...")

clf = RF( n_estimators = 100, verbose = True, n_jobs = -1 )
clf.fit( x_train, y_train )

p = clf.predict_proba( x_test )[:,1]
auc = AUC( y_test, p )
print ("AUC: {:.2%}".format( auc ))

# AUC: 50.28%

(40000, 29) (12000, 28)
LR...


D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 64 concurrent workers.


AUC: 45.27%
RF...
AUC: 72.55%


[Parallel(n_jobs=-1)]: Done  74 out of 100 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  74 out of 100 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=64)]: Done 100 out of 100 | elapsed:    0.0s finished


In [19]:
# meta_train = pd.DataFrame()
# meta_test = pd.DataFrame()
# meta_train['predictions'] = pd.Series(result_dict_lgb['oof'].reshape(-1))#.rank()
#seperate train rows which have been misclassified as test and use them as validation
import copy
train2 = copy.deepcopy(data)#.sort_values('CUST_UID')
train2["predictions"] = pd.Series(clf.predict_proba(data.drop( [ 'TARGET', 'CUST_UID', 'LABEL' ] + cat_cols+large_offset, axis = 1 ))[:,1].reshape(-1))
predictions_argsort = train2['predictions'].argsort()
#print(predictions_argsort)
train_sorted = train2.iloc[predictions_argsort]
# print(train_sorted.tail())
#select only trains set because we need to find train rows which have been misclassified as test set and use them for validation
train_sorted = train_sorted.loc[train_sorted.TARGET == 0]

#Why did I chose 0.7 as thereshold? just a hunch, but you should try different thresholds i.e 0.6, 0.8 and see the difference in validation score and please report back. :) 
train_as_test = train_sorted.loc[train_sorted.predictions > 0.7]
print(train_as_test.shape)
print(train_as_test.head())
# train_as_test.to_csv('adversarial_set_data.csv', index=False)
# #save the indices of the misclassified train rows to use as validation set
# adversarial_set_ids = train_as_test.index.values
# adversarial_set = pd.DataFrame(adversarial_set_ids, columns=['adversial_set_ids'])
# #save adversarial set index
# adversarial_set.to_csv('adversarial_set_ids.csv', index=False)
# print(adversarial_set.shape)
# adversarial_set.head()

[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  74 out of 100 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=64)]: Done 100 out of 100 | elapsed:    0.0s finished


(7, 53)
                               CUST_UID  LABEL  AGN_CNT_RCT_12_MON  \
50394  77cacef0e32745fca4c9e21846d2a6c1    0.0                -1.0   
47685  b397b06dd45e47c8b8374a7b967b49a1    0.0                42.0   
50899  afed25b29193433ab4b835e501ba00a7    1.0              2872.0   
47242  0d52778ec12944008627550ecfaf59c6    1.0                -1.0   
43359  2b0bb2e8a3f64c8780e103c3e5fc7ef7    0.0                -1.0   

       ICO_CUR_MON_ACM_TRX_TM  NB_RCT_3_MON_LGN_TMS_AGV  AGN_CUR_YEAR_AMT  \
50394                    -1.0                      -1.0              -1.0   
47685                    62.0                     318.7          612008.0   
50899                   332.0                    1042.0         5777058.3   
47242                    42.0                     472.0              -1.0   
43359                    12.0                      42.0              -1.0   

       AGN_CUR_YEAR_WAG_AMT  AGN_AGR_LATEST_AGN_AMT  ICO_CUR_MON_ACM_TRX_AMT  \
50394                  -1.0 